## AI-powered Contact Center Intake (AICCI) Agents - Memory/Router Design

 * Author:    Kyle Zarifsadr
 * Created:   January 2025

In [ ]:
from dotenv import load_dotenv
load_dotenv() 
import numpy as np
import pandas as pd
import json
import os
import openai
import random
import time
from datetime import datetime
from openai import AzureOpenAI
import psycopg
from sentence_transformers import SentenceTransformer
import autogen
from autogen import GroupChat
from autogen import GroupChatManager
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen import ConversableAgent, UserProxyAgent, config_list_from_json
from autogen.retrieve_utils import TEXT_FORMATS
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from autogen import AssistantAgent, ConversableAgent, UserProxyAgent
import re

# LLM

### Azure OpenAI

Reference: https://microsoft.github.io/autogen/0.2/docs/reference/agentchat/contrib/vectordb/pgvectordb

In [ ]:
llm_config = {
    "config_list": [
        {
            "model": "gpt-4o",
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_type": "azure",
            "base_url": os.getenv("OPENAI_API_BASE"),
            "api_version": os.getenv("API_VERSION"),
        },
    ],
    "temperature": 0.0,
    "timeout": 300,
}

In [ ]:
load_dotenv()  

api_key = os.getenv("OPENAI_API_KEY")
azure_endpoint = os.getenv("OPENAI_API_BASE")
api_version = os.getenv("API_VERSION")
model = "gpt-4o"
temprature = 0.0
top_p = 1.0

def get_response_client(template, text, temprature=temprature, top_p=top_p, model=model, azure_endpoint=azure_endpoint, api_key=api_key, api_version=api_version):
    time.sleep(1)

    client = AzureOpenAI(
        azure_endpoint = azure_endpoint, 
        api_key= api_key,  
        api_version= api_version
    )

    conversation = [
        {"role": "system", "content": template},
        {"role": "user", "content": text},
    ]

    response = client.chat.completions.create(
        model=model,
        temperature=temprature,
        top_p= top_p,
        max_tokens=4000,
        response_format={ "type": "json_object" },  #JSON mode
        messages=conversation
    )

    return response.choices[0].message.content

# Knowledge Base

### Simulated API Calls to PostgreSQL

In [ ]:
def get_db_connection():
    conn = psycopg2.connect(
        dbname="test",
        user="postgres",
        password="test_password",
        host="localhost",
        port="5433"
    )
    return conn

def make_request(url, clientKey):
    conn = get_db_connection()
    cur = conn.cursor()

    query = "SELECT response_data FROM guidelines WHERE url = %s AND clientKey = %s;"
    cur.execute(query, (url, clientKey))
    result = cur.fetchone()
    
    if result:
        response_data = result[0]
        print(f"Response from {url}: {json.dumps(response_data)}")
        return response_data
    #     store_request_response(url, params, response_data)
    # else:
    #     print(f"No data found for URL: {url}")
    #     store_request_response(url, params, {"error": "Not Found"})
    
    cur.close()
    conn.close()

### Test clientKey

In [ ]:
clientKey = 'Call Center Training 32 - Target Corporation'  # [ 'Call Center Training 32 - Target Corporation' ,'Call Center Training 31 - UPS', 'StarSpaces', 'BankCorp', 'HealthCarePlus', 'EduCare',]

In [ ]:
url = "/v6.0/Intake/guidelines"
guidelines = make_request(url, clientKey)

In [ ]:
url = "/v6.0/Intake/locations"
locations = make_request(url, clientKey)

In [ ]:
url = "/v6.0/Intake/GetMobileIssueTypesWithDefaults"
IssueTypes = make_request(url, clientKey)

In [ ]:
url = "/v6.0/ViolationQuestion/GetViolationQuestionAndAnswersForPlatformPackages"
questions = make_request(url, clientKey)

In [ ]:
## Should be added to the DB
implicated_parties = {"Affected Party":"","Perpetrator":"","Witness":"","Other":""}

# AGENTIC DESIGN

### 0. Router and Memory Agents

Design Consideration: 
Track past interactions and select which group chat should be involved next
 - metadata for each group chat 
 - agent priorities
 - memory access to past conversation
 - user intent
 - termination requirement for each group chat

In [ ]:
template_memory = """
Role: JSON Chat Memory: You will be given a new message and a JSON Chat Memory.
Your primary responsibility is to memorize the new message and the JSON Chat Memory.
You will take to action!

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

template_router = """
Role: You are a classification agent. 
Objective: You will be given the caller's new message, {{JSON Chat Memory}}, {{group_chat}} priorities, and applicable {{group_chat}}'s {{Termination Requirements}}. 
Your primary responsibility is to detect the the action id for the new message to route to by checking the {{JSON Chat Memory}} against the applicable {{Termination Requirements}} 
and determine the next {{group_chat}} only from the following {{group_chat}} options.

{
  "group_chat": 
  [
    {"id": "main_imminent_issue",
    "priority": 1, 
    "role": "This function is used when AI detects imminent issue",
    "termination_requirements": {
        "requirement_1": "presence of imminent issue event in context",
      }
    },
    {"id": "main_guidelines",
    "priority": 2, 
    "role": "This function is used when the JSON Chat Memory does not contain the customer's responses to the READ messages in guidelines/instructions.",
    "termination_requirements": {
        "requirement_1": "Answer to all READ Messages",
        "requirement_2": "Have identified the caller's intent to call, whether it is a new report or a follow-up."
      }
    },
    {"id": "main_locations",
    "priority": 3, 
    "role": "This function is used when the JSON Chat Memory does not contain the customer responses to the location indentification requirements.",
    "termination_requirements": {
        "requirement_1": "Respond to all location-related inquiries", 
        "requirement_2": "Identify the State, City, and Building details."
      }
    },
    {"id": "main_anonymous_mode",
    "priority": 4, 
    "role": "This function is used when the JSON Chat Memory does not contain the customer responses to the report mode requirements.",
    "termination_requirements": {
        "requirement_1": "The status of Report Mode is being collected and stored in memory.",
        "requirement_2":"if "Report Mode" in content: report_mode_value = content["Report Mode"]",
      }
    },
    {"id": "main_issue_questions",
    "priority": 5, 
    "role": "This function is used when input message is the transcription of the event.",
    "termination_requirements": {
        "requirement_1": "transcription of the report which is the primary reason for the call is in memory"
      }
    },
    {"id": "main_issue_questions_follow_up",
    "priority": 6, 
    "role": "This function is used when the JSON Chat Memory does not contain the customer responses to the issue questions",
    "termination_requirements": {
        "requirement_1": "Answer to all retrieved violation type questions."
      }
    },
    {"id": "main_implicated_parties",
    "priority": 7, 
    "role": "This function is used when the JSON Chat Memory does not contain the customer responses to the issue questions",
    "termination_requirements": {
        "requirement_1": "Information of implicated parties including their names",
        "requirement_2": "Information of implicated parties including their roles",
        "requirement_3": "Information of implicated parties including their job titles"
      }
    },
    {"id": "main_final_allegation",
    "priority": 8, 
    "role": "This function is used when the JSON Chat Memory does not contain a reevaluation of the "ViolationTypeId" after the initial one.",
    "termination_requirements": {
        "requirement_1": "Information of the presence of "ViolationTypeId" for the second time in memory after implicated parties",
      }
    },
    {"id": "main_report_review",
    "priority": 9, 
    "role": "This function is used when the JSON Chat Memory does not contain the report summary read by the reporter.",
    "termination_requirements": {
        "requirement_1": "Information of reviewing the summary of the report.",
      }
    },  
    {"id": "terminate_chat",
    "priority": 10, 
    "role": "This function is used when the JSON Chat Memory does contain the customer responses to the previous priorities.",
    "termination_requirements": {
        "requirement_1": "Answer to all questions"
      }
    }
  ]
}

If the {{Termination Requirements}} are satisfied, your detected action will be terminate_chat and you are Done!.

example: 

{"action":"main_issue_questions"}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

template_imminent_issue = """
At any point during the interview, AI detects imminent issue and flags report with priority with this message: {"Final Message":"Flag report with priority"}
At any point during the interview, (If suicide ideations deteted, transfer call to CS workflow) with this message: {"Final Message":"transfer call to CS workflow"}

Examples:
Threats of Violence: Reports of threats of physical harm or violence from coworkers, supervisors, or customers.
Harassment or Assault: Incidents of sexual harassment, physical assault, or severe bullying.
Medical Emergencies: Situations involving severe health issues such as heart attacks, severe allergic reactions, or injuries.
Unsafe Working Conditions: Immediate dangers due to unsafe conditions like hazardous materials, lack of safety equipment, or structural hazards.
Fire or Explosion: Reports of fires, explosions, or similar emergencies posing immediate threats to safety.
Active Shooter or Hostage Situation: Incidents involving an active shooter or hostage situation within the workplace.
Severe Psychological Distress: Cases of severe psychological distress or suicidal thoughts/intentions.

(If suicide ideations deteted, transfer call to CS workflow) with this message: "transfer call to CS workflow"

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [ ]:
with open("memory.json", "w") as json_file:
    json.dump([], json_file)

def load_chat_history():
    with open("memory.json", "r") as json_file:
        chat_history_data = json.load(json_file)
    return chat_history_data

chat_history = load_chat_history()
print("Initial chat history:", chat_history)

In [ ]:
memory_agent = ConversableAgent(
    name="memory_agent",
    system_message= f"{template_memory} + chat history: {chat_history}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

router_agent = ConversableAgent(
    name="router_agent",
    system_message= f"{template_router} + guidelines instructions: {guidelines}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

In [ ]:
def state_transition_router(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is memory_agent:
        return router_agent
    elif last_speaker is router_agent:
        return None

groupchat_router = autogen.GroupChat(
    agents=[memory_agent,
            router_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_router,
)

manager_router = autogen.GroupChatManager(groupchat=groupchat_router, llm_config=llm_config)

# # Test
# last_message = chat_history
# history = memory_agent.initiate_chat(recipient=manager_router, message=str(last_message))

### 1. Process Guidelines Data
Tagging and Parsing Approach


In [ ]:
template_process_guidelines = """
Extract text in JSON from text or HTML elements similar to the following:

{
  "OPENING": "Text from opening HTML element",
  "READ": {
    "READ_1": "1st Text to read",
    "READ_2": "2nd Text to read",
  },
  "DONT_READ": {
    "DONT_READ_1": "1st Text not to read",
    "DONT_READ_2": "2nd Text not to read",
  },
  "Other_Instructions": {
    "new_report": "Other instructions for new report",
    "follow_up": "Other instructions for follow-up"
  }
}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [ ]:
guidelines = get_response_client(template_process_guidelines, guidelines)
guidelines = json.loads(guidelines)
print(guidelines)

### 2. Imminent Issues

#### 2.a. Imminent Issues: Agents

In [ ]:
user_agent = ConversableAgent(
    name="user_agent",
    system_message= f"You are an assistant",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

imminent_issue_agent = ConversableAgent(
    name="imminent_issue_agent",
    system_message= f"Imminent Issues: {template_imminent_issue}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 2.b. Imminent Issues: Group Chat

In [ ]:
def state_transition_imminent_issue(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return imminent_issue_agent
    elif last_speaker is imminent_issue_agent:
        return None

groupchat_imminent_issue = autogen.GroupChat(
    agents=[user_agent,
            imminent_issue_agent
           ],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition_imminent_issue,
)

manager_imminent_issue = autogen.GroupChatManager(groupchat=groupchat_imminent_issue, llm_config=llm_config)

#### 2.c. Imminent Issues: Functions

In [ ]:
def main_imminent_issue():
    memory = read_chat_history()
    memory.append(new_message)

    last_message = json.dumps(memory)

    user_agent.initiate_chat(recipient=manager_imminent_issue, message=last_message, clear_history=False)
    messages_json = manager_imminent_issue.messages_to_string(manager_imminent_issue.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

### 3. Guidelines

#### 3.a. Guidelines: Agents

In [ ]:
template_guidelines = """
Role: Ethics and Compliance Contact Center Agent
Objective: You will be given the callers [previous_chat_history] and applicable [guidelines]. 
Your primary responsibility is to determine the call reasons "New Report"/ "Follow-Up" after gathering all answers to the [guidelines] READ messages in a valid JSON.

Don't repeat the content of the previous_chat_history.
Under no circumstances determine the call reasons if you don't have an answer to the READ messages/questions.
Your response should consist of only one question at a time retrieved from the READ messages/questions.
You will categorize the call as either a new report or a follow-up on an existing case.

Call Reasons:
New Report: The caller is calling to file a new report.
Follow-Up: The caller is calling to follow up on an existing case.

Steps to Determine Call Reason:
Always start with the "Opening" text from the [guidelines]. check if the "Opening" statement has already been used in the current session 
and then proceed to the next relevant READ messages.

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [ ]:
user_agent = ConversableAgent(
    name="user_agent",
    system_message= f"You are an assistant",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

guidelines_agent = ConversableAgent(
    name="guidelines_agent",
    system_message= f"Report Datetime: {template_guidelines} + guidelines: {guidelines}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 3.b. Guidelines: Group Chat

In [ ]:
def state_transition_guidelines(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return guidelines_agent
    elif last_speaker is guidelines_agent:
        return None

groupchat_guidelines = autogen.GroupChat(
    agents=[user_agent,
            guidelines_agent
           ],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition_guidelines,
)

manager_guidelines = autogen.GroupChatManager(groupchat=groupchat_guidelines, llm_config=llm_config)

#### 3.c. Guidelines: Functions

In [ ]:
def collect_json_data():
    follow_ups = {}
    while not follow_ups:
        user_input = input('Please provide data in JSON format (e.g. {"key": "value"} or {"key1": "value1", "key2": "value2"}): ')
        try:
            user_data = json.loads(user_input)
            if isinstance(user_data, dict):
                follow_ups.update(user_data)
            else:
                print("The JSON input must be an object (key-value pairs). Please try again.")
        except json.JSONDecodeError:
            print("Invalid JSON input. Please try again.")
    return follow_ups

def extract_questions_from_memory(memory):
    last_message = memory[-1]
    content = last_message["content"]

    try:
        content_dict = json.loads(content)
        return content_dict
    except json.JSONDecodeError:
        questions = [q.strip() + "?" if not q.strip().endswith("?") else q.strip() for q in content.split("\n") if q.strip()]
        questions_dict = {q: "" for q in questions}
        # return questions_dict
        return json.dumps(questions_dict)

def read_chat_history():
    try:
        with open("memory.json", "r") as json_file:
            try:
                return json.load(json_file)
            except json.JSONDecodeError:
                return []  
    except FileNotFoundError:
        return [] 

def extract_and_print_questions():
    memory = read_chat_history()
    questions = extract_questions_from_memory(memory)
    print(questions)

def write_chat_history(chat_history_data):
    seen = set()
    unique_messages = []

    for message in chat_history_data:
        message_str = json.dumps(message, sort_keys=True, ensure_ascii=False)  
        if message_str not in seen:
            seen.add(message_str)
            unique_messages.append(message)


    with open("memory.json", "w", encoding="utf-8") as json_file:
        json.dump(unique_messages, json_file, indent=4, ensure_ascii=False)

In [ ]:
def main_guidelines(new_message):
    memory = read_chat_history()
    memory.append(new_message)

    last_message = json.dumps(new_message)

    user_agent.initiate_chat(recipient=manager_guidelines, message=last_message, clear_history=False)
    messages_json = manager_guidelines.messages_to_string(manager_guidelines.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

### 4. Locations

#### 4.a. Locations: Agents

In [ ]:
template_locations = """
Retrieve the location of the event from [locations] by asking a few questions one by one about the state, city, etc., 
using [locations] data.

Do not repeat questions with the same context.

As soon as you detect the location from [locations], respond with a question to confirm "Case_CompanyLocation": "", 
and "Case_CompanyCity": "" , and  "Case_CompanyLocation":"" from the data package available in [locations]. 

Your final question would be to confirm the location of the event. 

Example: 

"Could you please confirm if the event took place in Austin, Texas, in Building D?"

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [ ]:
locations_agent = ConversableAgent(
    name="locations",
    system_message= f"locations: {locations} + {template_locations}",
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 4.b. Locations: Group Chat

In [ ]:
def state_transition_locations(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return locations_agent
    elif last_speaker is locations_agent:
        return None

groupchat_locations = autogen.GroupChat(
    agents=[user_agent,
            locations_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_locations,
)

manager_locations = autogen.GroupChatManager(groupchat=groupchat_locations, llm_config=llm_config)

#### 4.c. Locations: Functions

In [ ]:
def main_locations(new_message):
    memory = read_chat_history()
    memory.append(new_message)
   
    last_message = json.dumps(new_message)

    user_agent.initiate_chat(recipient=manager_locations, message=last_message, clear_history=False)
    messages_json = manager_locations.messages_to_string(manager_locations.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

### 5. Report Mode

In [ ]:
template_report_mode = """
Detect the report mode based on the answer you receive. Your answer should be 
a valid JSON with "Report Mode" as the key and the detected report mode, either "Anonymous" or "Non-Anonymous," as the value.

Example:
1. {"Report Mode":"Anonymous"}
2. {"Report Mode":"Non-Anonymous"}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

template_anonymous_mode = """You are reviewing reports submitted by employees at a company who wish to submit their report anonymously. Please read through the report and
only remove any of the person writing the reports personal information, such as their phone number, email address, name, location (such as
office number or desk/cubicle/work location which would identify the reporter), race, job title, length of employment, disabilities or gender.
Information about people they witnessed should remain. The report should retain as much of the original text as possible without giving identifying
details about the person reporting it.

Example1: "I am the only female/engineer in the office/location" should be removed.
Example2: "The colleague from Marketing, who often sits two desks away from me" should be changed to "The colleague from Marketing".
Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

anonymous_mode_agent = ConversableAgent(
    name="anonymous_mode",
    system_message= template_report_mode,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

def state_transition_anonymous_mode(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return anonymous_mode_agent
    elif last_speaker is anonymous_mode_agent:
        return None

groupchat_anonymous_mode = autogen.GroupChat(
    agents=[user_agent,
            anonymous_mode_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_anonymous_mode,
)

manager_anonymous_mode = autogen.GroupChatManager(groupchat=groupchat_anonymous_mode, llm_config=llm_config)

def collect_anonymous_mode():
    system_default_note = "Do you wish to provide your name today, or would you rather remain anonymous?"
    anonymous_mode = {system_default_note: ""}

    while not anonymous_mode[system_default_note]:
        user_input = input("{\"Do you wish to provide your name today, or would you rather remain anonymous?\": \"\"}): ")
        try:
            user_data = json.loads(user_input)
            anonymous_mode.update(user_data)
        except json.JSONDecodeError:
            print("Invalid JSON input. Please try again.")
    
    anonymous_mode_text = convert_json_to_code(json.dumps(anonymous_mode))
    
    return anonymous_mode_text

def main_anonymous_mode(collect_anonymous_mode): 
    user_agent.initiate_chat(recipient=manager_anonymous_mode, message=str(collect_anonymous_mode), clear_history=False)
    messages_json = manager_anonymous_mode.messages_to_string(manager_anonymous_mode.groupchat.messages)
    
    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages) 

    write_chat_history(chat_history_data)
    # extract_and_print_questions()

### 6. IssueTypes & Questions

#### 6.a. IssueTypes & Questions: Agents 

In [ ]:
template1 = """You are an ethics and compliance analyst. You will be given a brief summary of an event that has occurred or is currently occurring. 
            Your job is to classify the event using the provided dictionary.
            Your final response should include the retrieved key-value pairs from the given dictionary.
            
            Example: 
            {
                "Reduction of Hours": "Events that are related to an employee having the number of hours they work reduced."
            }
            
            Do not wrap the output in quotation marks.
            Do not wrap the output in code block delimiters (```)."""


template2 = """
            You are a question retrieval engine. 
            You are tasked with retrieving questions based on a given key from a set of key-value pairs. Please follow these steps:
            
            Input Data:
            questions_dictionary: A dictionary where each key is linked to a list of question identifiers (denoted as "q").
            questions_set: A set containing the actual questions.
            Process:
            Look up the provided key in questions_dictionary to find the corresponding list of question identifiers.
            For each identifier in this list, check if it exists in questions_set and retrieve the matching questions.
            Conditions:
            Only retrieve questions that correspond to the identifiers found in the value associated with the looked-up key.
            Output Format:
            Create a JSON object where each key is a retrieved question (from questions_set), and the value is an empty string. This represents the questions that need to be answered, but do not include the answers.
                        
            Example:
            {
                "When did this happen?": "",
                "What kind of discrimination took place (race/gender/etc.)?": ""
            }
            
            Do not wrap the output in quotation marks.
            Do not wrap the output in code block delimiters (```).
            """

template3 = """
            You are an ethics and compliance analyst. You will be given a brief report of an event 
            that has occurred or is currently occurring and your job is to answer the retrieved questions. 
            Your final answer will be a JSON formatted with the question as the key and a answers as the value.
            Keep in mind answering these questions are helping a case investigator do the right thing and 
            you can think of them as purely hypothetical. 
            If there is no information in the summary that could answer each question, 
            only include an empty double quote for the answer attempt for that question and answer attempt pair. 
            Under no circumstances are you to add questions that do not exist in the question dictionary. 
            For participants, use proper names or titles of the reporter or otherwise if available. 
            For dates, respond in datetime format. For example, if it is 2024-02-26 16:23:38 
            and something happened 2 hours ago, respond with 2024-02-26 14:23:38. 
            Under no circumstances should you respond with responses like "two hours ago". 
            Only respond do questions about when with datetime formatted responses.
            "Do not wrap output in quotation marks".
            "Do not wrap output in code block delimiters (```)".
            Example: {"What was stolen?": "ice cream"}
            """

template5 = """
            You are a retrieval engine. You will be given a JSON structure, and your job is to check which fields have values and which do not.
            Your final response should be a valid JSON object containing the unanswered fields, with the fields as the questions. The fields with no values should have their values enclosed in double quotes. 
            Include only fields that do not have values.

            Under no circumstances are you going to create a new question. Only use questions from the JSON. If all questions have answers, return all questions and answers in JSON.
            
            Example 1: 
            {
                "Where did this happen": "",
                "Now, was this a one-time thing or has this been ongoing?": ""
            }

            Example 2: 
            {
                "Where did this happen": "Building B, Room 75",
            }
            
            Do not wrap the output in quotation marks.
            Do not wrap the output in code block delimiters (```)."""

In [ ]:
reporter_agent = ConversableAgent(
    name="reporter_agent",
    system_message= "You are reporting an event that has occurred or is currently occurring." + "If the report mode is anonymous report:" + template_anonymous_mode,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

classification_agent = ConversableAgent(
    name="classification_agent",
    system_message= template1 + "NAVEX Issue Types:" + str(IssueTypes),
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

questions_retriever_agent = ConversableAgent(
    name="questions_retriever_agent",
    system_message= "Questions Dictionary:" + str(questions) + template2 + "If the report mode is anonymous report:" + template_anonymous_mode,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

questions_answering_agent = ConversableAgent(
    name="questions_answering_agent",
    system_message= template3,
    llm_config=llm_config,
    human_input_mode="NEVER",
)

unanswered_questions_agent = ConversableAgent(
    name="unanswered_questions_agent",
    system_message= template5,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 6.b. IssueTypes & Questions: Group Chat

In [ ]:
def state_transition_issue_questions(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is reporter_agent:
        return classification_agent
    elif last_speaker is classification_agent:
        return questions_retriever_agent
    elif last_speaker is questions_retriever_agent:
        return questions_answering_agent
    elif last_speaker is questions_answering_agent:
        return unanswered_questions_agent
    elif last_speaker is unanswered_questions_agent:
        return None

groupchat_issue_questions = autogen.GroupChat(
    agents=[reporter_agent,
            classification_agent, 
            questions_retriever_agent, 
            questions_answering_agent,
            unanswered_questions_agent
           ],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition_issue_questions,
)

manager_issue_questions = autogen.GroupChatManager(groupchat=groupchat_issue_questions, llm_config=llm_config)

#### 6.c. IssueTypes & Questions: Functions

In [ ]:
def convert_json_to_code(json_input):
    try:
        data = json.loads(json_input)
        code_lines = []

        for key, value in data.items():
            value_str = value if isinstance(value, str) else json.dumps(value)
            code_lines.append(f'{key} = {value_str}')

        return "\n".join(code_lines)
    
    except json.JSONDecodeError:
        return "Invalid JSON provided to the code conversion function."
        

def collect_transcription_data():
    system_default_note = "Now, in a sentence or two, please describe the primary reason for your call."
    transcription = {system_default_note: ""}

    while not transcription[system_default_note]:
        user_input = input("Please provide transcription text in JSON format (e.g. {\"Now, in a sentence or two, please describe the primary reason for your call.\": \"I want to start filing a report!\"}): ")
        try:
            user_data = json.loads(user_input)
            transcription.update(user_data)
        except json.JSONDecodeError:
            print("Invalid JSON input. Please try again.")
    
    transcription_text = convert_json_to_code(json.dumps(transcription))
    
    return transcription_text

In [ ]:
def main_issue_questions(collect_transcription_data): 
    reporter_agent.initiate_chat(recipient=manager_issue_questions, message=str(collect_transcription_data), clear_history=False)
    messages_json = manager_issue_questions.messages_to_string(manager_issue_questions.groupchat.messages)
    
    chat_history_data = read_chat_history()

    new_messages = json.loads(messages_json)  
    chat_history_data.extend(new_messages) 

    write_chat_history(chat_history_data)
    extract_and_print_questions()


def main_issue_questions_follow_up(new_message):
    memory = read_chat_history()
    memory.append(new_message)

    last_message = json.dumps(new_message)

    user_agent.initiate_chat(recipient=manager_issue_questions, message=last_message, clear_history=False)
    messages_json = manager_issue_questions.messages_to_string(manager_issue_questions.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

### 7. Implicated Parties

#### 7.a. Implicated Parties: Agents

In [ ]:
template_implicated_parties = """
You are an ethics and compliance analyst. You will be given a brief report of an event 
that has occurred or is currently occurring and your job is to ask one or maximum two questions at a time to extract 
the names, last names, job titles, and confirm spelling of participant names involved and assign a role. 
If there is no information in the report that could be assigned to a role, 
only include an empty double quote for that roles.

role's example = ["Affected Party", "Perpetrator", "Witness", "Other"]

Steps to Follow:

- Extract the first name and last name of participants involved.
- Assign each participant a role from the roles_list.
- Ask for and confirm the job title of each participant.
- If the report lacks sufficient information to assign a role, gather detailed information to accurately determine the role.
- Use proper names to identify participants. If a participant is described using pronouns such as "me", "my", or "I", leave the participant's name blank and only provide single quotes "".
- Output: Your final answer should be formatted in JSON, containing the role, participant's first name, last name, and job title.

Engage in a step-by-step conversation to gather detailed information about the participants involved in the case using a polite/professional tone.
Ensure you have the correct names and job titles of the individuals involved.
Confirm the spelling of their names.
Document the information accurately in JSON format.

If the report is in anonymous mode, don't ask questions about the reporter's identification, as questions may reveal their identity.

Ask one or maximum two questions at a time to maintain the conversational flow and ask one or maximum two follow-ups based on responses.

Example: 

Event: John witnessed, Emma stole my backpack

{"Let's confirm spelling of his name":"'J' as in 'jump', 'o' as in 'go', 'h' as in 'hat', 'n' as in 'nose', 'D' as in 'dog', 'o' as in 'go', 'e' as in 'elephant'.",
 "And would you please tell me what his job title is?": "Software Engineer"
} 

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

In [ ]:
implicated_parties_agent = ConversableAgent(
    name="participants_identifier_agent",
    system_message= template_implicated_parties + "Case Participants List: " + str(implicated_parties) + "If the report mode is anonymous report:" + template_anonymous_mode,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

#### 7.b. Implicated Parties: Group Chat

In [ ]:
def state_transition_implicated_parties(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return implicated_parties_agent
    elif last_speaker is implicated_parties_agent:
        return questions_answering_agent
    elif last_speaker is questions_answering_agent:
        return unanswered_questions_agent
    elif last_speaker is unanswered_questions_agent:
        return None

groupchat_implicated_parties = autogen.GroupChat(
    agents=[user_agent,
            implicated_parties_agent,
            questions_answering_agent,
            unanswered_questions_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_implicated_parties,
)

manager_implicated_parties = autogen.GroupChatManager(groupchat=groupchat_implicated_parties, llm_config=llm_config)

#### 7.c. Implicated Parties: Functions

In [ ]:
def main_implicated_parties(new_message):
    memory = read_chat_history()
    memory.append(new_message)
    
    last_message = json.dumps(new_message)

    user_agent.initiate_chat(recipient=manager_implicated_parties, message=last_message, clear_history=False)
    messages_json = manager_implicated_parties.messages_to_string(manager_implicated_parties.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

### 8. Final Allegation

In [ ]:
def state_transition_final_allegation(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is reporter_agent:
        return classification_agent
    elif last_speaker is classification_agent:
        return None

groupchat_final_allegation = autogen.GroupChat(
    agents=[reporter_agent,
            classification_agent
           ],
    messages=[],
    max_round=20,
    speaker_selection_method=state_transition_final_allegation,
)

manager_final_allegation = autogen.GroupChatManager(groupchat=groupchat_final_allegation, llm_config=llm_config)

def main_final_allegation():
    memory = read_chat_history()
    last_message = json.dumps(memory)

    reporter_agent.initiate_chat(recipient=manager_final_allegation, message=last_message, clear_history=False)
    messages_json = manager_final_allegation.messages_to_string(manager_final_allegation.groupchat.messages)

    new_messages = json.loads(messages_json)
    final_allegation = new_messages[-1]
    memory.append(final_allegation)
    
    write_chat_history(memory)
    # extract_and_print_questions()

### 9. Report Review

In [ ]:
template_report_review = """
You are an ethics and compliance report analyst. Summarize the report in detail for the reporter with a professional tone. 
This summarization is the final read and has to be representative of the actual events that have happened, 
including the report of the primary reason for your call, content of the questions and answers, when it happened, and who was involved. 
Under no circumstances should you make up any information; you should only summarize in detail.

Respond in JSON format.

Example:

{"Kindly allow me a few moments to review the summary of the report for you.
So at 2 p.m. on December 24th, 2015, the caller overheard Craig and Sue discussing Craig's personal financial woes in Sue's room, room 216, 
and his need for money to purchase a home. Shortly after the conversation, the caller saw Sue heading towards Craig's office with a check. 
When the caller inquired about what Sue held, Sue confirmed that she was holding a Christmas gift for someone. 
The caller observed Sue hand Craig the check in his office. Craig embraced Sue and cried. 
The caller does not know if anyone else overheard Craig in Sue's conversation or witnessed Sue providing Craig the check. 
Now, after work, the caller saw that Craig posted on Facebook, Christmas miracles do happen, received $2,000 for my home today. 
Although the name of the issuer of the check was concealed, the caller believes this was the check that Sue gave Craig. Now, per company policy, 
employees cannot accept gifts of any amount from residents under any circumstances.":""}

Do not wrap the output in quotation marks.
Do not wrap the output in code block delimiters (```).
"""

report_review_agent = ConversableAgent(
    name="report_review_agent",
    system_message= template_report_review,
    llm_config=llm_config,
    human_input_mode= "NEVER",
)

def state_transition_report_review(last_speaker, groupchat):
    messages = groupchat.messages

    if last_speaker is user_agent:
        return report_review_agent
    elif last_speaker is report_review_agent:
        return None

groupchat_report_review = autogen.GroupChat(
    agents=[user_agent,
            report_review_agent
           ],
    messages=[],
    max_round=10,
    speaker_selection_method=state_transition_report_review,
)

manager_report_review = autogen.GroupChatManager(groupchat=groupchat_report_review, llm_config=llm_config)

def main_report_review():
    memory = read_chat_history()
    
    user_agent.initiate_chat(recipient=manager_report_review, message=str(memory), clear_history=False)
    messages_json = manager_report_review.messages_to_string(manager_report_review.groupchat.messages)

    new_messages = json.loads(messages_json)
    memory.extend(new_messages)

    write_chat_history(memory)
    extract_and_print_questions()

## Final Call Simulation

In [ ]:
def terminate_chat():
    memory = read_chat_history()
    new_message = {"final_message": "Thanks for filling this report"}
    memory.append(new_message)

    with open("memory.json", "w") as test_file:
        json.dump(memory, test_file)

    chat_history_data = read_chat_history()
    write_chat_history(chat_history_data)
    print(new_message)

In [ ]:
def detect_action():
    memory = read_chat_history()
    new_message = collect_json_data()
    memory.append(new_message)

    last_message = json.dumps(memory)
    history = memory_agent.initiate_chat(recipient=manager_router, message=last_message, clear_history=False)
    message = history.chat_history[-1]["content"]
    last_message = json.loads(message)
    action = last_message.get("action", None)

    if action == "main_anonymous_mode":
        new_message = collect_anonymous_mode()
        return action, new_message

    elif action == "main_issue_questions":
        new_message = collect_transcription_data()
        return action, new_message

    return action, new_message
    

def process_action(action):
    if action == "main_imminent_issue":
        return main_imminent_issue()
    elif action == "main_guidelines":
        return main_guidelines(new_message)
    elif action == "main_locations":
        return main_locations(new_message)
    elif action == "main_anonymous_mode":
        return main_anonymous_mode(new_message)
    elif action == "main_issue_questions":
        return main_issue_questions(new_message)
    elif action == "main_issue_questions_follow_up":
        return main_issue_questions_follow_up(new_message)
    elif action == "main_implicated_parties":
        return main_implicated_parties(new_message)
    elif action == "main_final_allegation":
        return main_final_allegation()
    elif action == "main_report_review":
        return main_report_review()
    elif action == "terminate_chat":
        return terminate_chat()
    else:
        print(f"Invalid action: {action}")
        return {"error": "Invalid action"}


if __name__ == "__main__":
    while True:
        action, new_message = detect_action()
        process_action(action)
        if action == "terminate_chat":
            break

In [ ]:
with open("memory.json", "r") as json_file:
    chat_history_guidelines = json.load(json_file)
print(json.dumps(chat_history_guidelines, indent=3))